In [1]:
# lr:0.001
# anneal_rate:0.9
# batch_size:32
# clip_norm:50
# num_epochs:5
# alpha:250
# beta:0
# max_beta:1
# step_beta:0.002
# anneal_iter:40000
# kl_anneal_iter:2000
# print_iter:100
# save_iter:5000
# num_workers:4

In [2]:
!pip install -q torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu116.html
!pip install -q dive-into-graphs

In [3]:
!pip install -q toolz
!pip install -q wandb

In [4]:
%load_ext autoreload
%autoreload 2
import os
import json
import argparse
import pickle 

import numpy as np
import pandas as pd
import torch

from molecule_optimizer.externals.fast_jtnn.datautils import SemiMolTreeFolder, SemiMolTreeFolderTest
from molecule_optimizer.runner.semi_jtvae import SemiJTVAEGeneratorPredictor
from torch_geometric.data import DenseDataLoader

import rdkit

lg = rdkit.RDLogger.logger() 
lg.setLevel(rdkit.RDLogger.CRITICAL)

import warnings
warnings.filterwarnings("ignore")

import math

In [5]:
conf = json.load(open("training/configs/rand_gen_zinc250k_config_dict.json"))

In [6]:
csv = pd.read_csv("ZINC_310k.csv")

In [7]:
smiles = csv['SMILES']

In [8]:
smiles = smiles[:60000]

In [9]:
labels = torch.tensor(csv['QED'][:60000]).float()

In [10]:
# if 'runner.xml' not in os.listdir("."):
#     runner = SemiJTVAEGeneratorPredictor(smiles)
#     with open('runner.xml', 'wb') as f:
#         pickle.dump(runner, f)

In [11]:
if 'runner_20.xml' not in os.listdir("."):
    runner = SemiJTVAEGeneratorPredictor(smiles)
    with open('runner_20.xml', 'wb') as f:
        pickle.dump(runner, f)

In [12]:
# with open('runner.xml', 'rb') as f:
#     runner = pickle.load(f)

In [13]:
with open('runner_20.xml', 'rb') as f:
    runner = pickle.load(f)

In [14]:
runner.get_model(
    "rand_gen",
    {
        "hidden_size": conf["model"]["hidden_size"],
        "latent_size": conf["model"]["latent_size"],
        "depthT": conf["model"]["depthT"],
        "depthG": conf["model"]["depthG"],
        "label_size": 1,
        "label_mean": float(torch.mean(labels)),
        "label_var": float(torch.var(labels)),
    },
)

In [15]:
labels = runner.get_processed_labels(labels)
preprocessed = runner.processed_smiles

In [16]:
# N_TEST = 10000
N_TEST = 200
VAL_FRAC = 0.05

In [17]:
perm_id=np.random.permutation(len(labels))
X_train = preprocessed[perm_id[N_TEST:]]
L_train = torch.tensor(labels.numpy()[perm_id[N_TEST:]])


X_test = preprocessed[perm_id[:N_TEST]]
L_test = torch.tensor(labels.numpy()[perm_id[:N_TEST]])

In [18]:
val_cut = math.floor(len(X_train) * VAL_FRAC)

In [19]:
X_Val = X_train[:val_cut]
L_Val = L_train[:val_cut]

X_train = X_train[val_cut :]
L_train = L_train[val_cut :]

In [20]:
X_Val = X_train[:val_cut]
L_Val = L_train[:val_cut]

X_train = X_train[val_cut :]
L_train = L_train[val_cut :]

In [21]:
train_loader = SemiMolTreeFolder(
    X_train,
    L_train,
    runner.vocab,
    conf["batch_size"],
    label_pct=0.05,
    num_workers=conf["num_workers"],
)

In [22]:
test_loader = SemiMolTreeFolderTest(
    X_test,
    L_test,
    runner.vocab,
    conf["batch_size"],
    num_workers=conf["num_workers"],
)

In [23]:
batch_size_val = math.floor(len(X_Val) / 10)

val_loader = SemiMolTreeFolderTest(
    X_Val,
    L_Val,
    runner.vocab,
    batch_size_val,
    num_workers=conf["num_workers"],
)

In [ ]:
print("Training model...")
runner.train_gen_pred(
    loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    load_epoch=0,
    lr=conf["lr"],
    anneal_rate=conf["anneal_rate"],
    clip_norm=conf["clip_norm"],
    num_epochs=conf["num_epochs"],
    alpha=conf["alpha"],
    beta=conf["beta"],
    max_beta=conf["max_beta"],
    step_beta=conf["step_beta"],
    anneal_iter=conf["anneal_iter"],
    kl_anneal_iter=conf["kl_anneal_iter"],
    print_iter=100,
    save_iter=conf["save_iter"],
)

Training model...
Model #Params: 4732K
[Train][100] Alpha: 25.855, Beta: 0.000, Loss: 185.83, KL: 88.31, MAE: 0.11605, Word Loss: 93.89, Topo Loss: 25.57, Assm Loss: 8.96, Pred Loss: 0.86, Word: 26.94, Topo: 79.60, Assm: 54.69, PNorm: 101.80, GNorm: 50.00
[Train][200] Alpha: 97.049, Beta: 0.000, Loss: 112.03, KL: 114.33, MAE: 0.09732, Word Loss: 59.12, Topo Loss: 14.13, Assm Loss: 8.29, Pred Loss: 0.61, Word: 47.17, Topo: 90.21, Assm: 59.58, PNorm: 106.20, GNorm: 50.00
[Train][300] Alpha: 45.178, Beta: 0.000, Loss: 100.00, KL: 166.14, MAE: 0.08528, Word Loss: 51.28, Topo Loss: 12.61, Assm Loss: 8.26, Pred Loss: 0.47, Word: 54.77, Topo: 91.20, Assm: 59.23, PNorm: 109.49, GNorm: 50.00
[Train][400] Alpha: 65.815, Beta: 0.000, Loss: 90.68, KL: 198.94, MAE: 0.08198, Word Loss: 47.03, Topo Loss: 10.52, Assm Loss: 7.81, Pred Loss: 0.42, Word: 58.13, Topo: 92.60, Assm: 61.20, PNorm: 112.49, GNorm: 50.00
[Train][500] Alpha: 38.085, Beta: 0.000, Loss: 88.47, KL: 235.73, MAE: 0.07544, Word Loss: 

In [ ]:
print("Training model...")
runner.train_gen_pred_supervised(
    loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    load_epoch=0,
    lr=conf["lr"],
    anneal_rate=conf["anneal_rate"],
    clip_norm=conf["clip_norm"],
    num_epochs=conf["num_epochs"],
    alpha=conf["alpha"],
    beta=conf["beta"],
    max_beta=conf["max_beta"],
    step_beta=conf["step_beta"],
    anneal_iter=conf["anneal_iter"],
    kl_anneal_iter=conf["kl_anneal_iter"],
    print_iter=100,
    save_iter=conf["save_iter"],
)